In [1]:
from utils.load_results import *
from utils.plot_helpers import *

import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
plt.style.use('default')
import torch
from utils.analysis_from_interaction import *
from language_analysis_local import TopographicSimilarityConceptLevel, encode_target_concepts_for_topsim
import os
if not os.path.exists('analysis'):
    os.makedirs('analysis')
#import plotly.express as px

In [2]:
datasets = ['(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)']
n_values = [4, 8, 16, 4, 8, 4]
n_attributes = [3, 3, 3, 4, 4, 5]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [3]:
datasets = ['(4,4)',]
n_values = [4,]
n_attributes = [4,]
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [2]:
datasets = ['(3,4)',]
n_values = [4,]
n_attributes = [3,]
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [3]:
context_unaware = False # whether original or context_unaware simulations are evaluated
if context_unaware:
    setting = 'context_unaware'
else:
    setting = 'standard'

### Determine vocab size

In [4]:
from collections import Counter

In [5]:
# go through all datasets
for i, d in enumerate(datasets):
    # select first run
    path_to_run = paths[i] + '/' + str(setting) +'/' + str(0) + '/'
    path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
    path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
    interaction = torch.load(path_to_interaction_train)
    print(path_to_interaction_train)
    
    messages = interaction.message.argmax(dim=-1)
    messages = [msg.tolist() for msg in messages]
    all_symbols = [symbol for message in messages for symbol in message]
    symbol_counts = Counter(all_symbols)
    print(symbol_counts)
    print(len(symbol_counts))

results/(3,4)_game_size_10_vsf_3/standard/0/interactions/train/epoch_300/interaction_gpu0
Counter({0: 1807, 6: 779, 2: 582, 8: 537, 11: 483, 15: 478, 4: 454, 3: 435, 7: 393, 5: 373, 13: 359, 10: 198, 14: 142, 1: 126, 9: 82})
15
results/(3,8)_game_size_10_vsf_3/standard/0/interactions/train/epoch_300/interaction_gpu0
Counter({0: 11662, 25: 4555, 2: 2113, 12: 2050, 15: 1955, 14: 1894, 21: 1705, 4: 1553, 23: 1524, 24: 1500, 3: 1296, 20: 1272, 7: 1243, 5: 1162, 27: 1092, 10: 1042, 18: 988, 19: 969, 16: 940, 6: 927, 1: 906, 11: 898, 22: 831, 8: 754, 9: 589, 13: 494, 17: 425, 26: 309})
28
results/(3,16)_game_size_10_vsf_3/standard/0/interactions/train/epoch_300/interaction_gpu0
Counter({0: 83302, 47: 18222, 12: 13188, 37: 10467, 29: 10386, 14: 9187, 10: 9041, 51: 9012, 20: 8702, 8: 7622, 39: 6992, 34: 6854, 5: 6842, 46: 6470, 33: 6397, 28: 6390, 4: 6173, 30: 5326, 49: 5178, 35: 4978, 41: 4963, 16: 4866, 43: 4844, 1: 4646, 26: 4539, 13: 4495, 23: 4401, 36: 4382, 19: 4335, 6: 4263, 21: 4091, 4

### See which messages are produced for which concepts

In [19]:
datasets = ['(3,16)']
n_values = [16]
n_attributes = [3]
n_epochs = 300
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [20]:
context_unaware = False # whether original or context_unaware simulations are evaluated
if context_unaware:
    setting = 'context_unaware'
else:
    setting = 'standard'

In [21]:
# go through all datasets
for i, d in enumerate(datasets):
    # select first run
    path_to_run = paths[i] + '/' + str(setting) +'/' + str(0) + '/'
    path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
    path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
    interaction = torch.load(path_to_interaction_train)
    print(path_to_interaction_train)

    messages = interaction.message.argmax(dim=-1)
    messages = [msg.tolist() for msg in messages]
    sender_input = interaction.sender_input
    print(sender_input.shape)
    n_targets = int(sender_input.shape[1]/2)
    # get target objects and fixed vectors to re-construct concepts
    target_objects = sender_input[:, :n_targets]
    target_objects = k_hot_to_attributes(target_objects, n_values[i])
    # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
    (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
    concepts = list(zip(objects, fixed))

    # get distractor objects to re-construct context conditions
    distractor_objects = sender_input[:, n_targets:]
    distractor_objects = k_hot_to_attributes(distractor_objects, n_values[i])
    context_conds = retrieve_context_condition(objects, fixed, distractor_objects)

    # get random qualitative samples
    #fixed_index = random.randint(0, n_attributes[i]-1) # define a fixed index for the concept
    #n_fixed = random.randint(1, n_attributes[i]) # how many fixed attributes?
    n_fixed = 1
    #fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attributes are fixed
    fixed_indices = [0]
    #fixed_value = random.randint(0, n_values[i]-1) # define a fixed value for this index
    fixed_values = random.choices(range(0, n_values[i]), k=n_fixed)
    fixed_values = [10, 8, 11]
    print(n_fixed, fixed_indices, fixed_values)
    #index_threshold = 20000 # optional: define some index threshold to make sure that examples are not taken from the beginning of training
    # TODO: adapt this loop such that multiple indices can be fixed
    all_for_this_concept = []
    for idx, (t_objects, t_fixed) in enumerate(concepts):
        #if sum(t_fixed) == 1 and t_fixed[fixed_index] == 1:# and idx > index_threshold:
        if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
            for t_object in t_objects:
                if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                    all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                    fixed = t_fixed
    if len(all_for_this_concept) > 0:
        #sample = random.sample(all_for_this_concept, 20)
        sample = all_for_this_concept
        column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
        df = pd.DataFrame(sample, columns=column_names)
        df.to_csv('analysis/quali_' + str(d) + '_' + str(setting) + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv', index=False)
        print('saved ' + 'analysis/quali_' + str(d) + '_' + str(setting) + '_' + str(sample[0][1]) + ',' + str(fixed) + 'all.csv')
    else:
        raise ValueError("sample for dataset " + str(d) + " could not be generated")

results/(3,16)_game_size_10_vsf_3/standard/0/interactions/train/epoch_300/interaction_gpu0
torch.Size([83302, 20, 48])
1 [0] [10, 8, 11]
saved analysis/quali_(3,16)_standard_[10.  7.  6.],[1. 0. 0.]all.csv


In [5]:
df = pd.read_csv('analysis/quali_(3,4)_standard_[0. 2. 1.],[1. 1. 1.].csv')
df

,game_nr,object,fixed indices,context condition,message
0,884,[0. 2. 1.],[1. 1. 1.],0,"[2, 10, 3, 0]"
1,744,[0. 2. 1.],[1. 1. 1.],1,"[2, 10, 3, 0]"
2,1216,[0. 2. 1.],[1. 1. 1.],0,"[2, 10, 3, 0]"
3,1252,[0. 2. 1.],[1. 1. 1.],1,"[2, 10, 3, 0]"
4,637,[0. 2. 1.],[1. 1. 1.],2,"[2, 10, 3, 0]"
5,120,[0. 2. 1.],[1. 1. 1.],0,"[2, 10, 3, 0]"
6,1035,[0. 2. 1.],[1. 1. 1.],2,"[2, 10, 3, 0]"
7,653,[0. 2. 1.],[1. 1. 1.],0,"[2, 10, 3, 0]"
8,1573,[0. 2. 1.],[1. 1. 1.],1,"[2, 10, 3, 0]"
9,196,[0. 2. 1.],[1. 1. 1.],1,"[2, 10, 3, 0]"


In [4]:
df = pd.read_csv('analysis\quali_(3,4)_standard_[0. 2. 1.],[1. 1. 1.].csv')

# Get the context conditions
context_conditions = df['context condition'].unique()

# Iterate over each context condition
for context in context_conditions:
    print(f"Context Condition: {context}")
    print("Unique Messages:")
    
    # Filter the DataFrame for the current context condition
    context_df = df[df['context condition'] == context]
    
    # Get the unique messages for the current context condition
    unique_messages = context_df['message'].unique()
    
    # Print each unique message
    for message in unique_messages:
        print(message)
    
    print()

Context Condition: 0
Unique Messages:
[2, 10, 3, 0]

Context Condition: 1
Unique Messages:
[2, 10, 3, 0]

Context Condition: 2
Unique Messages:
[2, 10, 3, 0]



In [7]:
df = pd.read_csv('analysis/quali_(4,8)_standard_[7. 3. 1. 0.],[1. 1. 1. 1.]all.csv')
df

,game_nr,object,fixed indices,context condition,message
0,6176,[7. 3. 1. 0.],[1. 1. 1. 1.],1,"[13, 19, 24, 27, 0]"
1,6176,[7. 3. 1. 0.],[1. 1. 1. 1.],1,"[13, 19, 24, 27, 0]"
2,6176,[7. 3. 1. 0.],[1. 1. 1. 1.],1,"[13, 19, 24, 27, 0]"
3,6176,[7. 3. 1. 0.],[1. 1. 1. 1.],1,"[13, 19, 24, 27, 0]"
4,6176,[7. 3. 1. 0.],[1. 1. 1. 1.],1,"[13, 19, 24, 27, 0]"
...,...,...,...,...,...
295,137695,[7. 3. 1. 0.],[1. 1. 1. 1.],1,"[13, 19, 24, 27, 0]"
296,137695,[7. 3. 1. 0.],[1. 1. 1. 1.],1,"[13, 19, 24, 27, 0]"
297,137695,[7. 3. 1. 0.],[1. 1. 1. 1.],1,"[13, 19, 24, 27, 0]"
298,137695,[7. 3. 1. 0.],[1. 1. 1. 1.],1,"[13, 19, 24, 27, 0]"


In [8]:
df = pd.read_csv('analysis/quali_(4,8)_standard_[7. 3. 1. 0.],[1. 1. 1. 1.]all.csv')

# Get the context conditions
context_conditions = df['context condition'].unique()

# Iterate over each context condition
for context in context_conditions:
    print(f"Context Condition: {context}")
    print("Unique Messages:")
    
    # Filter the DataFrame for the current context condition
    context_df = df[df['context condition'] == context]
    
    # Get the unique messages for the current context condition
    unique_messages = context_df['message'].unique()
    
    # Print each unique message
    for message in unique_messages:
        print(message)
    
    print()

Context Condition: 1
Unique Messages:
[13, 19, 24, 27, 0]

Context Condition: 2
Unique Messages:
[3, 19, 9, 27, 0]
[13, 19, 24, 27, 0]

Context Condition: 3
Unique Messages:
[3, 19, 9, 27, 0]
[3, 21, 9, 27, 0]

Context Condition: 0
Unique Messages:
[13, 19, 24, 27, 0]



In [12]:
df = pd.read_csv('analysis/quali_(5,4)_standard_[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.]all.csv')
df

,game_nr,object,fixed indices,context condition,message
0,3328,[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.],0,"[2, 12, 5, 10, 7, 0]"
1,3328,[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.],0,"[2, 12, 5, 10, 7, 0]"
2,3328,[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.],0,"[2, 12, 5, 10, 7, 0]"
3,3328,[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.],0,"[2, 12, 5, 10, 7, 0]"
4,3328,[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.],0,"[2, 12, 5, 10, 7, 0]"
...,...,...,...,...,...
405,74561,[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.],4,"[2, 12, 5, 14, 3, 0]"
406,74561,[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.],4,"[2, 12, 5, 14, 3, 0]"
407,74561,[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.],4,"[2, 12, 5, 14, 3, 0]"
408,74561,[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.],4,"[2, 12, 5, 14, 3, 0]"


In [13]:
df = pd.read_csv('analysis/quali_(5,4)_standard_[3. 2. 2. 0. 2.],[1. 1. 1. 1. 1.]all.csv')

# Get the context conditions
context_conditions = df['context condition'].unique()

# Iterate over each context condition
for context in context_conditions:
    print(f"Context Condition: {context}")
    print("Unique Messages:")
    
    # Filter the DataFrame for the current context condition
    context_df = df[df['context condition'] == context]
    
    # Get the unique messages for the current context condition
    unique_messages = context_df['message'].unique()
    
    # Print each unique message
    for message in unique_messages:
        print(message)
    
    print()

Context Condition: 0
Unique Messages:
[2, 12, 5, 10, 7, 0]
[2, 12, 5, 14, 3, 0]

Context Condition: 3
Unique Messages:
[2, 5, 5, 10, 7, 0]
[2, 12, 5, 10, 7, 0]
[2, 12, 5, 14, 3, 0]
[2, 5, 5, 14, 3, 0]
[2, 5, 5, 10, 12, 0]

Context Condition: 4
Unique Messages:
[2, 5, 5, 10, 7, 0]
[2, 12, 5, 14, 3, 0]
[2, 12, 5, 10, 12, 0]

Context Condition: 1
Unique Messages:
[2, 12, 5, 14, 3, 0]
[2, 12, 5, 10, 7, 0]

Context Condition: 2
Unique Messages:
[2, 12, 5, 14, 3, 0]
[2, 12, 5, 10, 7, 0]



### Generic Concepts

In [7]:
df = pd.read_csv('analysis/quali_(3,4)_standard_[0. 3. 0.],[1. 0. 0.]all.csv')
df

,game_nr,object,fixed indices,context condition,message
0,173,[0. 3. 0.],[1. 0. 0.],0,"[2, 11, 5, 0]"
1,173,[0. 1. 1.],[1. 0. 0.],0,"[2, 11, 5, 0]"
2,173,[0. 0. 2.],[1. 0. 0.],0,"[2, 11, 5, 0]"
3,173,[0. 1. 0.],[1. 0. 0.],0,"[2, 11, 5, 0]"
4,173,[0. 3. 1.],[1. 0. 0.],0,"[2, 11, 5, 0]"
5,173,[0. 2. 3.],[1. 0. 0.],0,"[2, 11, 5, 0]"
6,173,[0. 1. 3.],[1. 0. 0.],0,"[2, 11, 5, 0]"
7,173,[0. 0. 0.],[1. 0. 0.],0,"[2, 11, 5, 0]"
8,173,[0. 3. 3.],[1. 0. 0.],0,"[2, 11, 5, 0]"
9,173,[0. 3. 2.],[1. 0. 0.],0,"[2, 11, 5, 0]"


In [8]:
df = pd.read_csv('analysis/quali_(3,4)_standard_[0. 3. 0.],[1. 0. 0.]all.csv')

# Get the context conditions
context_conditions = df['context condition'].unique()

# Iterate over each context condition
for context in context_conditions:
    print(f"Context Condition: {context}")
    print("Unique Messages:")
    
    # Filter the DataFrame for the current context condition
    context_df = df[df['context condition'] == context]
    
    # Get the unique messages for the current context condition
    unique_messages = context_df['message'].unique()
    
    # Print each unique message
    for message in unique_messages:
        print(message)
    
    print()

Context Condition: 0
Unique Messages:
[2, 11, 5, 0]
[11, 9, 5, 0]



In [12]:
df = pd.read_csv('analysis/quali_(4,8)_standard_[7. 0. 0. 7.],[1. 0. 0. 0.]all.csv')
df

,game_nr,object,fixed indices,context condition,message
0,6534,[7. 0. 0. 7.],[1. 0. 0. 0.],0,"[24, 24, 24, 22, 0]"
1,6534,[7. 0. 6. 2.],[1. 0. 0. 0.],0,"[24, 24, 24, 22, 0]"
2,6534,[7. 4. 2. 5.],[1. 0. 0. 0.],0,"[24, 24, 24, 22, 0]"
3,6534,[7. 4. 2. 6.],[1. 0. 0. 0.],0,"[24, 24, 24, 22, 0]"
4,6534,[7. 0. 1. 3.],[1. 0. 0. 0.],0,"[24, 24, 24, 22, 0]"
...,...,...,...,...,...
85,138488,[7. 0. 0. 0.],[1. 0. 0. 0.],0,"[24, 24, 24, 22, 0]"
86,138488,[7. 5. 3. 0.],[1. 0. 0. 0.],0,"[24, 24, 24, 22, 0]"
87,138488,[7. 0. 1. 4.],[1. 0. 0. 0.],0,"[24, 24, 24, 22, 0]"
88,138488,[7. 1. 1. 4.],[1. 0. 0. 0.],0,"[24, 24, 24, 22, 0]"


In [13]:
df = pd.read_csv('analysis/quali_(4,8)_standard_[7. 0. 0. 7.],[1. 0. 0. 0.]all.csv')

# Get the context conditions
context_conditions = df['context condition'].unique()

# Iterate over each context condition
for context in context_conditions:
    print(f"Context Condition: {context}")
    print("Unique Messages:")
    
    # Filter the DataFrame for the current context condition
    context_df = df[df['context condition'] == context]
    
    # Get the unique messages for the current context condition
    unique_messages = context_df['message'].unique()
    
    # Print each unique message
    for message in unique_messages:
        print(message)
    
    print()

Context Condition: 0
Unique Messages:
[24, 24, 24, 22, 0]



In [17]:
df = pd.read_csv('analysis/quali_(5,4)_standard_[3. 3. 0. 2. 2.],[1. 0. 0. 0. 0.]all.csv')
df

,game_nr,object,fixed indices,context condition,message
0,6781,[3. 3. 0. 2. 2.],[1. 0. 0. 0. 0.],0,"[7, 14, 14, 14, 3, 0]"
1,6781,[3. 3. 0. 2. 0.],[1. 0. 0. 0. 0.],0,"[7, 14, 14, 14, 3, 0]"
2,6781,[3. 2. 2. 2. 3.],[1. 0. 0. 0. 0.],0,"[7, 14, 14, 14, 3, 0]"
3,6781,[3. 2. 0. 0. 3.],[1. 0. 0. 0. 0.],0,"[7, 14, 14, 14, 3, 0]"
4,6781,[3. 3. 0. 3. 3.],[1. 0. 0. 0. 0.],0,"[7, 14, 14, 14, 3, 0]"
...,...,...,...,...,...
75,72523,[3. 3. 2. 2. 3.],[1. 0. 0. 0. 0.],0,"[7, 14, 14, 14, 3, 0]"
76,72523,[3. 1. 2. 0. 0.],[1. 0. 0. 0. 0.],0,"[7, 14, 14, 14, 3, 0]"
77,72523,[3. 2. 2. 2. 1.],[1. 0. 0. 0. 0.],0,"[7, 14, 14, 14, 3, 0]"
78,72523,[3. 2. 0. 1. 2.],[1. 0. 0. 0. 0.],0,"[7, 14, 14, 14, 3, 0]"


In [18]:
df = pd.read_csv('analysis/quali_(5,4)_standard_[3. 3. 0. 2. 2.],[1. 0. 0. 0. 0.]all.csv')

# Get the context conditions
context_conditions = df['context condition'].unique()

# Iterate over each context condition
for context in context_conditions:
    print(f"Context Condition: {context}")
    print("Unique Messages:")
    
    # Filter the DataFrame for the current context condition
    context_df = df[df['context condition'] == context]
    
    # Get the unique messages for the current context condition
    unique_messages = context_df['message'].unique()
    
    # Print each unique message
    for message in unique_messages:
        print(message)
    
    print()

Context Condition: 0
Unique Messages:
[7, 14, 14, 14, 3, 0]



In [22]:
df = pd.read_csv('analysis/quali_(3,16)_standard_[10.  7.  6.],[1. 0. 0.]all.csv')
df

,game_nr,object,fixed indices,context condition,message
0,17810,[10. 7. 6.],[1. 0. 0.],0,"[35, 4, 33, 0]"
1,17810,[10. 1. 12.],[1. 0. 0.],0,"[35, 4, 33, 0]"
2,17810,[10. 7. 0.],[1. 0. 0.],0,"[35, 4, 33, 0]"
3,17810,[10. 7. 12.],[1. 0. 0.],0,"[35, 4, 33, 0]"
4,17810,[10. 7. 2.],[1. 0. 0.],0,"[35, 4, 33, 0]"
...,...,...,...,...,...
85,66422,[10. 1. 12.],[1. 0. 0.],0,"[35, 7, 33, 0]"
86,66422,[10. 10. 0.],[1. 0. 0.],0,"[35, 7, 33, 0]"
87,66422,[10. 3. 10.],[1. 0. 0.],0,"[35, 7, 33, 0]"
88,66422,[10. 9. 8.],[1. 0. 0.],0,"[35, 7, 33, 0]"


In [23]:
df = pd.read_csv('analysis/quali_(3,16)_standard_[10.  7.  6.],[1. 0. 0.]all.csv')

# Get the context conditions
context_conditions = df['context condition'].unique()

# Iterate over each context condition
for context in context_conditions:
    print(f"Context Condition: {context}")
    print("Unique Messages:")
    
    # Filter the DataFrame for the current context condition
    context_df = df[df['context condition'] == context]
    
    # Get the unique messages for the current context condition
    unique_messages = context_df['message'].unique()
    
    # Print each unique message
    for message in unique_messages:
        print(message)
    
    print()

Context Condition: 0
Unique Messages:
[35, 4, 33, 0]
[35, 7, 33, 0]
[46, 7, 33, 0]

